In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [2]:
# random.sample(range(0, len(image_names)), 5)

In [3]:
image_names = get_png_names("../data/MEX/")#[0:1000]
# image_names = get_png_names("../pooling/data/MEX2/")
# image_indices = random.sample(range(0, len(image_names)), 1000)
# image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../../pooling/data/migration_data.json" , image_names)

2066 municipalities.


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .0001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

butler = geozoom_handler(attn_model, 
                         device, 
                         criterion, 
                         attn_optimizer, 
                         num_thresholds = 4,
                         reduction_percent = .80,
                         convergence_dims = (224, 224),
                         change_bounds = (-50, 50),
                         perc_change_thresh = .65, 
                         plot = False, 
                         v = False)

In [5]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [6]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [7]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  1239
Num validation:  827


In [8]:
# for i,o in train_dl:
#     print(i)
# #     print(load_inputs(i[0]).shape, o)

In [ ]:
butler.train_attn_model(train_dl, val_dl)

Epoch               Training Loss       Validation Loss     Threshold           % within bounds     


/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Epoch: 0            2025.4786           1800.0305           3                   
Epoch: 1            1964.2925           1746.2211           3                   
Epoch: 2            1929.3816           1778.7184           3                   
Epoch: 3            1905.6071           1712.4589           3                   
Epoch: 4            1885.3108           1743.4792           3                   0.5851493139628733  
Epoch: 5            1866.2623           1736.6288           3                   0.5205811138014528  
Epoch: 6            1852.9089           1785.3613           3                   0.5205811138014528  
Epoch: 7            1839.0129           1733.8759           3                   0.5706214689265536  
Epoch: 8            1830.8726           1770.0016           3                   0.5754640839386602  
Epoch: 9            1823.8553           1762.042            3                   0.606133979015335   
Epoch: 10           1818.3416           1705.7518           3         

In [14]:
butler.train_fc_model(train_dl, val_dl, outside_estimator = 'rf')

Random Forest MAE:  1455.3926666666669


In [ ]:
# image_names

In [19]:
butler.plot_attn_map(impath = '../data/MEX/484007070/pngs/484007070_2010_all_box484007070_MAY.png')


<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>